In [16]:
import pandas as pd

In [17]:
df = pd.read_csv('Du_lieu_nguoi_di_lam_crop.csv')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762 entries, 0 to 761
Columns: 141 entries, Tong12 to Q12_Rate15
dtypes: float64(25), int64(98), object(18)
memory usage: 839.5+ KB


In [19]:
# Lấy danh sách tất cả cột object
object_cols = df.select_dtypes(include="object").columns

# Duyệt từng cột và chuẩn hóa chuỗi
for col in object_cols:
    df[col] = df[col].astype(str).str.lower().str.strip()


In [20]:
df.columns = df.columns.str.lower()

In [ ]:
# Danh sách các cột cần gán -1 nếu không được hướng nghiệp
cols_q1 = ['q1a_namtrainjob', 'q1c_cer', 'q1d_recurjob', 'q1e_lifeuse']

# Điều kiện: q1_trainjob == 0  → không được hướng nghiệp
mask = df['q1_trainjob'] == 2

# Gán -2 cho các cột trên
df.loc[mask, cols_q1] = -2


In [22]:
cols = ['q1a_namtrainjob', 'q1c_cer', 'q1d_recurjob', 'q1e_lifeuse']

df[['q1_trainjob'] + cols].groupby('q1_trainjob').apply(lambda x: x.isna().sum())



C:\Users\Admin\AppData\Local\Temp\ipykernel_97824\3493881174.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[['q1_trainjob'] + cols].groupby('q1_trainjob').apply(lambda x: x.isna().sum())


,q1_trainjob,q1a_namtrainjob,q1c_cer,q1d_recurjob,q1e_lifeuse
q1_trainjob,,,,,
1,0,260,408,393,393
2,0,0,0,0,0


In [23]:
df.loc[:, cols] = df.loc[:, cols].fillna(-1) # Gán -1 cho các giá trị null trong các cột này, ý nghĩa là quên hoặc không trả lời


In [24]:
df.dropna(inplace=True)

In [25]:
for col in df.columns:
    null_count = df[col].isnull().sum()
    if null_count > 0:
        print(f"{col}: {null_count} null")

In [26]:
import pandas as pd

# 1) Nhóm nghề
groups = {
    "Nhóm kỹ thuật": "kt",
    "Nhóm nghiên cứu": "nc",
    "Nhóm nghệ thuật": "nt",
    "Nhóm xã hội": "xh",
    "Nhóm quản lý": "ql",
    "Nhóm nghiệp vụ": "nv",
}

# 2) DataFrame nhị phân rỗng
bin_df = pd.DataFrame()

# 3) Chỉ lấy các cột bắt đầu bằng 'Q7_Requi'
skill_cols = [c for c in df.columns if c.startswith("q7_requi")]

for col in skill_cols:
    skill_name = col.replace("q7_", "").lower()   # ví dụ: requi1_kheo_leo
    
    # ép dữ liệu cột này thành string, thay NaN bằng ""
    col_clean = df[col].fillna("").astype(str)
    
    # 4) Tạo 6 cột nhị phân cho mỗi nhóm nghề
    for g_vn, g_code in groups.items():
        new_col = f"{skill_name}_{g_code}"
        # dùng str.contains thay cho lambda để tránh lỗi float
        bin_df[new_col] = col_clean.str.contains(g_vn).astype(int)

# 5) Gộp lại
df_final = pd.concat([df, bin_df], axis=1)

df_final.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_97824\1846543230.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bin_df[new_col] = col_clean.str.contains(g_vn).astype(int)
C:\Users\Admin\AppData\Local\Temp\ipykernel_97824\1846543230.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bin_df[new_col] = col_clean.str.contains(g_vn).astype(int)
C:\Users\Admin\AppData\Local\Temp\ipykernel_97824\1846543230.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

,tong12,p1q2_typeschool,p1q3_typeclass,p1q5_gender,p1q6_ethn,v9,p1q8_afloseschool,p1q8a_job_afloseschool,p1q9_job_current,p1q10_chanjob,...,requi17_lam_viec_voi_ho_so_du_lieu_nt,requi17_lam_viec_voi_ho_so_du_lieu_xh,requi17_lam_viec_voi_ho_so_du_lieu_ql,requi17_lam_viec_voi_ho_so_du_lieu_nv,requi18_lam_viec_ti_mi_chi_tiet_kt,requi18_lam_viec_ti_mi_chi_tiet_nc,requi18_lam_viec_ti_mi_chi_tiet_nt,requi18_lam_viec_ti_mi_chi_tiet_xh,requi18_lam_viec_ti_mi_chi_tiet_ql,requi18_lam_viec_ti_mi_chi_tiet_nv
0,4.400000,0,0,0,0,36,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
1,4.600000,0,1,0,0,43,0,0,4,1,...,0,0,0,0,0,0,0,0,0,0
2,3.800000,0,0,1,0,35,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2.266667,0,1,0,0,43,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.533333,0,1,0,0,39,0,2,2,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
bin_df.head

<bound method NDFrame.head of      requi1_kheo_leo_kt  requi1_kheo_leo_nc  requi1_kheo_leo_nt  \
0                     0                   0                   0   
1                     0                   0                   0   
2                     0                   0                   0   
3                     0                   0                   0   
4                     0                   0                   0   
..                  ...                 ...                 ...   
757                   0                   0                   0   
758                   0                   0                   0   
759                   0                   0                   0   
760                   0                   0                   0   
761                   0                   0                   0   

     requi1_kheo_leo_xh  requi1_kheo_leo_ql  requi1_kheo_leo_nv  \
0                     0                   0                   0   
1                     0        

In [28]:


# 1. Map tên nhóm nghề (đúng y như trong cột q7_requi*, có dấu & chữ thường)
groups = {
    "nhóm kỹ thuật": "kt",
    "nhóm nghiên cứu": "nc",
    "nhóm nghệ thuật": "nt",
    "nhóm xã hội": "xh",
    "nhóm quản lý": "ql",
    "nhóm nghiệp vụ": "nv",
}

# 2. Lấy 18 cột gốc của Câu 7
q7_cols = [c for c in df_final.columns if c.lower().startswith("q7_requi")]
print("Các cột C7 gốc:", q7_cols)

# 3. Từ mỗi cột q7_requi* → fill vào 6 cột nhị phân tương ứng
for col in q7_cols:
    # ví dụ: 'q7_requi1_kheo_leo' -> 'requi1_kheo_leo'
    base = col.lower().replace("q7_", "")

    # chuẩn hóa nội dung về chữ thường, thay NaN bằng chuỗi rỗng
    col_clean = df_final[col].fillna("").str.lower()

    for g_name, g_code in groups.items():
        new_col = f"{base}_{g_code}"   # ví dụ: requi1_kheo_leo_kt
        df_final[new_col] = col_clean.str.contains(g_name).astype(int)

# 4. Kiểm tra nhanh xem dữ liệu đã khác 0 chưa
print(df_final[[c for c in df_final.columns if c.startswith("requi1_kheo_leo")]].sum())
print(df_final[[c for c in df_final.columns if c.startswith("requi2_sang_tao")]].sum())


Các cột C7 gốc: ['q7_requi1_kheo_leo', 'q7_requi2_sang_tao', 'q7_requi3_dinh_huong', 'q7_requi4_nghien_cuu', 'q7_requi5_phan_tich', 'q7_requi6_viet', 'q7_requi7_tuong_tuong', 'q7_requi8_am_nhac', 'q7_requi9_bieu_dien_nghe_thuat', 'q7_requi10_noi', 'q7_requi11_nghe', 'q7_requi12_thuyet_trinh', 'q7_requi13_lam_viec_voi_nguoi_khac', 'q7_requi14_gay_anh_huong', 'q7_requi15_thuyet_phuc', 'q7_requi16_vuot_kho_khan_nguy_hiem', 'q7_requi17_lam_viec_voi_ho_so_du_lieu', 'q7_requi18_lam_viec_ti_mi_chi_tiet']
requi1_kheo_leo_kt    357
requi1_kheo_leo_nc     48
requi1_kheo_leo_nt    378
requi1_kheo_leo_xh    100
requi1_kheo_leo_ql     28
requi1_kheo_leo_nv     44
dtype: int64
requi2_sang_tao_kt    289
requi2_sang_tao_nc    168
requi2_sang_tao_nt    353
requi2_sang_tao_xh    127
requi2_sang_tao_ql     58
requi2_sang_tao_nv     38
dtype: int64


In [29]:
df_final.to_csv('Du_lieu_nguoi_di_lam_final.csv', index=False)

In [30]:
tq = pd.read_csv('Du_lieu_nguoi_di_lam_final.csv')
print(tq.columns.tolist())

['tong12', 'p1q2_typeschool', 'p1q3_typeclass', 'p1q5_gender', 'p1q6_ethn', 'v9', 'p1q8_afloseschool', 'p1q8a_job_afloseschool', 'p1q9_job_current', 'p1q10_chanjob', 'p1q11_mostsuc', 'p1q12_fjob', 'p1q13_mjob', 'p1q14_megr9', 'p1q15_hsub', 'p1q16_lsub', 'p1q16_likesub', 'p1q16_dislikesub', 'q1_trainjob', 'q1a_namtrainjob', 'q1c_cer', 'q1d_recurjob', 'q1e_lifeuse', 'q2_care1', 'q2_care2', 'q2_care3', 'q2_care4', 'q2_care5', 'q2_care6', 'q3_where1', 'q3_where2', 'q3_where3', 'q3_where4', 'q3_where5', 'q3_where6', 'q3_where7', 'q3_where8', 'q4_act1', 'q4_act2', 'q4_act3', 'q4_act4', 'q4_act5', 'q4_act6', 'q4_act7', 'q4_act8', 'q4_act9', 'q4_act10', 'q4_act11', 'q5_affect1', 'q5_affect2', 'q5_affect3', 'q5_affect4', 'q5_affect5', 'q5_affect6', 'q5_affect7', 'q5_affect8', 'q5_affect9', 'q5_affect10', 'q5_affect11', 'q5_affect12', 'q5_affect13', 'q5_affect14', 'q5_affect15', 'q5_affect16', 'q5_affect17', 'q5_affect18', 'q5_affect19', 'q5_affect20', 'q5_affect21', 'q6_likeen', 'q6_likere', 'q